## Creating CSV file

After we get the data from the Github archive, we need to filter the scope that we need to built the network, specificcaly **PullRequestReview** and **CommitCommentEvent** events, and concat this data in a csv file to improve the manipulation of this data.

In [1]:
import os
import csv
import gzip
import json
import pandas as pd
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
origin = os.getcwd()  + '/data'
print(origin)

/home/jupyter/notebooks/projeto/data


In [ ]:
COLUMNS = ["COMMENT_ID", "COMMIT_ID", "URL", "AUTHOR", "CREATED_AT", "BODY"]
PATH_TO_CSV = os.path.join(os.getcwd(), "colaboration_comments.csv")
for root, d, files in os.walk(origin):
    print(files)
    for i, f in enumerate(files):
        comments_list = []
        comments = pd.DataFrame(columns = COLUMNS)
        sys.stderr.write("%d\r" % i)
        path = os.path.join(root, f)
        if path.endswith(".gz"):
            try:
                for line in gzip.open(path).readlines():
                    event = json.loads(line)
                    if "PullRequestReview" in event["type"] or "CommitCommentEvent" in event["type"]:
                        comments_list.append([event["payload"]["comment"]["id"],
                                              event["payload"]["comment"]["commit_id"],
                                              event["payload"]["comment"]["html_url"],
                                              event["payload"]["comment"]["user"]["login"],
                                              event["payload"]["comment"]["created_at"],
                                              event["payload"]["comment"]["body"]])
            except EOFError:
                continue
        df = pd.DataFrame(comments_list, columns=COLUMNS)
        comments = comments.append(df, ignore_index=True)
        comments.to_csv(PATH_TO_CSV, mode="a", header=(not os.path.exists(PATH_TO_CSV)), index=False)

In [4]:
data = pd.read_csv(PATH_TO_CSV)

In [5]:
data


,COMMENT_ID,COMMIT_ID,URL,AUTHOR,CREATED_AT,BODY
0,22396772,47c1cd384b3deb0427e7e546e8d6b4346a65d816,https://github.com/selfhub/selfhub/pull/124#di...,zindlerb,2015-01-01T00:00:08Z,:sparkles: :pray: :sparkles:
1,22396775,a12155068a4feb03bf6b149b60830154ff52db0f,https://github.com/tsuru/tsuru/pull/1044#discu...,msabramo,2015-01-01T00:00:53Z,> Can you remove this empty line?\r\n\r\nDone.
2,9131174,f2c7da93b6607d2990335ad4c0b5f1b5c26f5a83,https://github.com/GoldenLions/GitHubScout/com...,myclamm,2015-01-01T00:00:55Z,Thanks man. That's from week 4 of Hack Reactor...
3,22396779,4c31e9b700de04c01f07c3f7d12afccbbf47237e,https://github.com/selfhub/selfhub/pull/125#di...,zindlerb,2015-01-01T00:01:44Z,"_.reduce would be more elegant, but your call"
4,22396780,2a0664c165aa3d1a4fec06d720d4ba393fefc7ae,https://github.com/emberjs/data/pull/2649#disc...,rwjblue,2015-01-01T00:01:54Z,Tiny nit-pick: can you name this something mea...
...,...,...,...,...,...,...
635297,9969493,d430ad5c96dc5a20ac90443cae1d534e21478c2d,https://github.com/AGV-IIT-KGP/git-sandbox/com...,rings360,2015-02-28T09:59:16Z,Asar Sunny <ring2rings93@gmail.com>
635298,9969494,3432d7bf3b10b57287979176ba569e414ad3af1e,https://github.com/veswill3/braintree-angular/...,jeffcarp,2015-02-28T09:59:16Z,Thank you for doing this! Would you mind if I ...
635299,25556037,062e3cb4c8aa6b3f2a8c338404bf699dfb8d2b6d,https://github.com/Robpol86/colorclass/pull/7#...,msabramo,2015-02-28T09:59:42Z,Done.
635300,25556038,062e3cb4c8aa6b3f2a8c338404bf699dfb8d2b6d,https://github.com/Robpol86/colorclass/pull/7#...,msabramo,2015-02-28T09:59:52Z,Done.
